In [1]:
import pandas as pd
from datetime import datetime
from utility.helpers import DataManager
import visualization

datamanager = DataManager('provisioned', 4)

In [2]:
# Load the data
data = datamanager.load_data()
# Convert timestamp to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [3]:
# Visualize data
visualization.visualize_data(data)
data

,instance_id,timestamp,query_count,runtime,bytes_scanned
0,4,2024-02-29 23:00:00,1,1546,1315.0
1,4,2024-03-01 00:00:00,8334,38989865,270649152.0
2,4,2024-03-01 01:00:00,9573,18817973,251629065.0
3,4,2024-03-01 02:00:00,11037,24512813,219990412.0
4,4,2024-03-01 03:00:00,9899,29100500,227740532.0
...,...,...,...,...,...
1405,4,2024-04-28 12:00:00,11124,20230903,149600146.0
1406,4,2024-04-28 13:00:00,10772,12816265,147060663.0
1407,4,2024-04-28 14:00:00,10122,19675888,146689262.0
1408,4,2024-04-28 15:00:00,10153,15793307,147946283.0


In [6]:
# Split into training and test data:
# Following the approach in the paper (p. 132), for a cluster with N weeks of data, 
# the first train-test split includes N-2 weeks for training and the following week 
# for testing. The second train-test split contains the first N-1 weeks for training 
# and the following week for testing, representing a scenario of re-training a model 
# each week and forecasting for the next week

train1, test1, train2, test2 = datamanager.train_test_split(data)

# Check the timestamps of train and test data
print(f"train1 start: {train1.iloc[0]['timestamp']}")
print(f"train1 end: {train1.iloc[-1]['timestamp']}")

print(f"test1 start: {test1.iloc[0]['timestamp']}")
print(f"test1 end: {test1.iloc[-1]['timestamp']}")


print(f"train2 start: {train2.iloc[0]['timestamp']}")
print(f"train2 end: {train2.iloc[-1]['timestamp']}")

print(f"test2 start: {test2.iloc[0]['timestamp']}")
print(f"test2 end: {test2.iloc[-1]['timestamp']}")


train1 start: 2024-02-29 23:00:00
train1 end: 2024-04-14 23:00:00
test1 start: 2024-04-15 00:00:00
test1 end: 2024-04-21 23:00:00
train2 start: 2024-02-29 23:00:00
train2 end: 2024-04-21 23:00:00
test2 start: 2024-04-22 00:00:00
test2 end: 2024-04-27 23:00:00


In [5]:
# Train baseline models

In [6]:
# Test baseline models and calculate error metrics

In [7]:
# visualize first prediction
visualization.visualize_prediction(train1, test1, prediction1)

# visualize first prediction
visualization.visualize_prediction(train2, test2, prediction2)

NameError: name 'prediction1' is not defined